In [1]:
push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/models"));
push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/processing"));
push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/util"));
ENV["COLUMNS"] = 200;

In [2]:
using Dates
using JuMP
using CSV
using DataFrames
using LinearAlgebra

In [3]:
using BedsData
using ForecastData
using GeographicData

In [4]:
import PatientAllocationResults

In [5]:
using PatientAllocation

In [6]:
states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI"]

start_date = Date(2020, 5, 1)
end_date   = Date(2020, 5, 30)

pct_beds_available = 0.25
travel_threshold_hours = 4.0
hospitalized_days = 14;

In [7]:
N = length(states);
T = (end_date - start_date).value + 1;

In [8]:
forecast_admitted = forecast(
    states, start_date, end_date,
    level=:state,
    source=:ihme,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
);

In [9]:
forecast_initial = forecast(
    states, start_date-Dates.Day(1), start_date-Dates.Day(1),
    level=:state,
    source=:ihme,
    forecast_type=:active,
    patient_type=:regular,
    bound_type=:mean,
)[:];

In [10]:
forecast_discharged = forecast(
    states, start_date-Dates.Day(hospitalized_days), start_date-Dates.Day(1),
    level=:state,
    source=:ihme,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
)
forecast_discharged = hcat(forecast_discharged, zeros(Float32, N, T - hospitalized_days));

In [20]:
beds = n_beds(states, bed_type=:all, pct_beds_available=pct_beds_available);
adj = adjacencies(states, level=:state, source=:google, threshold=travel_threshold_hours);

In [12]:
model = patient_allocation(
    beds,
    forecast_initial,
    forecast_admitted,
    forecast_discharged,
    adj,
    hospitalized_days=hospitalized_days,
    send_new_only=true,
    sendreceive_switch_time=3,
    min_send_amt=10,
    smoothness_penalty=0.001,
    setup_cost=0,
    sent_penalty=0,
    verbose=true
)
sent = value.(model[:sent])
println("termination status: ", termination_status(model))
println("solve time: ", round(solve_time(model), digits=3), "s")
println("objective function value: ", round(objective_value(model), digits=3))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 7904 rows, 7360 columns and 197380 nonzeros
Model fingerprint: 0x04b8db8d
Model has 580 SOS constraints
Variable types: 4360 continuous, 0 integer (0 binary)
Semi-Variable types: 3000 continuous, 0 integer
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-03, 1e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [4e-02, 1e+04]
Presolve removed 3330 rows and 2978 columns
Presolve time: 0.35s
Presolved: 7694 rows, 5942 columns, 109239 nonzeros
Presolved model has 492 SOS constraint(s)
Variable types: 4352 continuous, 1590 integer (1590 binary)
Found heuristic solution: objective 49335.747608
Found heuristic solution: objective 44994.698058

Root relaxation: objective 8.402908e+03, 7227 iterations, 0.90 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work


In [13]:
results = PatientAllocationResults.results_all(sent, beds, forecast_initial, forecast_admitted, forecast_discharged, states, start_date, hospitalized_days);

In [14]:
println("Total overflow: ", results.total_overflow)
println("Average load: ", results.average_load)

Total overflow: 8399.866149902344
Average load: 0.5506364827962167


In [15]:
results.summary_table

,state,total_sent,total_received,overflow,average_load
,String,Float64,Float64,Float64,Float64
1,CT,809.276,0.0,769.123,0.592566
2,DE,0.0,235.298,0.0,0.639091
3,MA,240.0,1977.97,0.0,0.626058
4,MD,2326.99,0.0,181.623,0.630969
5,ME,0.0,331.66,0.0,0.253065
6,NH,0.0,0.0,0.0,0.717196
7,NJ,2997.0,1668.41,7449.12,0.588492
8,NY,0.0,8146.68,0.0,0.398497
9,PA,5656.35,0.0,0.0,0.441473


In [16]:
results.sent_matrix_table

,state,CT,DE,MA,MD,ME,NH,NJ,NY,PA,RI
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,CT,0.0,0.0,0.0,0.0,312.192,0.0,0.0,497.084,0.0,0.0
2,DE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,MA,0.0,0.0,0.0,0.0,-2.15408e-11,0.0,0.0,240.0,0.0,0.0
4,MD,0.0,143.757,0.0,0.0,0.0,0.0,0.0,2183.23,0.0,0.0
5,ME,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,NH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,NJ,0.0,91.541,1977.97,0.0,0.0,0.0,0.0,907.486,0.0,20.0
8,NY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,PA,0.0,0.0,0.0,0.0,0.0,0.0,1477.47,4178.89,0.0,0.0


In [17]:
println("First day:")
filter(row -> row.day == start_date, results.complete_table)

First day:


,state,day,sent,received,new_patients,active_patients,capacity,overflow,load,sent_to,sent_from
,String,Date,Float64,Float64,Float32,Float64,Float32,Float64,Float64,Any,Any
1,CT,2020-05-01,321.51,0.0,321.51,2132.26,1840.25,613.52,1.15868,"[(""ME"", 302.192), (""NY"", 19.3181)]",[]
2,DE,2020-05-01,0.0,166.643,34.1128,422.936,506.75,0.0,0.834605,[],"[(""MD"", 143.757), (""NJ"", 22.8852)]"
3,MA,2020-05-01,0.0,657.667,327.558,2830.91,3997.75,0.0,0.708125,[],"[(""NJ"", 657.667)]"
4,MD,2020-05-01,216.532,0.0,308.294,2193.84,2228.75,181.623,0.984337,"[(""DE"", 143.757), (""NY"", 72.7743)]",[]
5,ME,2020-05-01,0.0,302.192,4.4961,336.079,668.0,0.0,0.503112,[],"[(""CT"", 302.192)]"
6,NH,2020-05-01,0.0,0.0,32.9998,181.678,584.5,0.0,0.310826,[],[]
7,NJ,2020-05-01,922.172,0.0,922.172,7083.11,4553.25,3452.03,1.55562,"[(""DE"", 22.8852), (""MA"", 657.667), (""NY"", 231.62), (""RI"", 10.0)]",[]
8,NY,2020-05-01,0.0,323.713,1051.65,7359.89,10406.2,0.0,0.707257,[],"[(""CT"", 19.3181), (""MD"", 72.7743), (""NJ"", 231.62)]"
9,PA,2020-05-01,0.0,0.0,674.188,4603.58,7980.25,0.0,0.576872,[],[]


In [18]:
s = "NJ"
filter(row -> row.state == s, results.complete_table)

,state,day,sent,received,new_patients,active_patients,capacity,overflow,load,sent_to,sent_from
,String,Date,Float64,Float64,Float32,Float64,Float32,Float64,Float64,Any,Any
1,NJ,2020-05-01,922.172,0.0,922.172,7083.11,4553.25,3452.03,1.55562,"[(""DE"", 22.8852), (""MA"", 657.667), (""NY"", 231.62), (""RI"", 10.0)]",[]
2,NJ,2020-05-02,915.841,0.0,915.841,6000.03,4553.25,2362.62,1.31775,"[(""DE"", 22.8852), (""MA"", 657.667), (""NY"", 225.289), (""RI"", 10.0)]",[]
3,NJ,2020-05-03,910.815,0.0,910.815,4961.67,4553.25,1319.23,1.0897,"[(""DE"", 22.8852), (""MA"", 662.641), (""NY"", 225.289)]",[]
4,NJ,2020-05-04,248.174,0.0,913.432,4620.32,4553.25,315.24,1.01473,"[(""DE"", 22.8852), (""NY"", 225.289)]",[]
5,NJ,2020-05-05,0.0,0.0,940.136,4553.25,4553.25,0.0,1.0,[],[]
6,NJ,2020-05-06,0.0,0.0,938.541,4456.06,4553.25,0.0,0.978655,[],[]
7,NJ,2020-05-07,0.0,0.0,866.922,4225.75,4553.25,0.0,0.928073,[],[]
8,NJ,2020-05-08,0.0,31.2825,844.234,3949.92,4553.25,0.0,0.867495,[],"[(""PA"", 31.2825)]"
9,NJ,2020-05-09,0.0,31.2825,816.379,3626.83,4553.25,0.0,0.796537,[],"[(""PA"", 31.2825)]"


In [19]:
results.sent_to

Dict{String,Array{String,1}} with 10 entries:
  "NH" => String[]
  "CT" => ["ME", "NY"]
  "RI" => ["ME", "NJ", "NY"]
  "MA" => ["NY"]
  "ME" => String[]
  "NY" => String[]
  "NJ" => ["DE", "MA", "NY", "RI"]
  "DE" => String[]
  "PA" => ["NJ", "NY"]
  "MD" => ["DE", "NY"]